In [6]:
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 1.3 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
df = pd.read_excel('/workspaces/Data_Scinece_Project/train.xlsx')

In [3]:
df.head()

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,target
0,-70,-61,-66,-53,-51,-63,-82,-57,-76,-78,-66,-66,-61,-59,-73,-75,-63,-77,B37
1,-77,-74,-71,-76,-65,-63,-66,-52,-55,-75,-72,-75,-74,-61,-64,-63,-53,-63,B61
2,-53,-38,-55,-66,-62,-62,-65,-70,-62,-52,-56,-53,-66,-68,-72,-60,-68,-77,A19
3,-72,-62,-59,-65,-65,-65,-78,-82,-83,-59,-84,-60,-64,-83,-69,-72,-95,-73,A22
4,-67,-69,-65,-63,-59,-53,-70,-72,-71,-60,-61,-57,-54,-76,-61,-66,-71,-80,A33


In [11]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from kmodes.kprototypes import KPrototypes
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


# Extract features and target
features = df.drop('target', axis=1)
target = df['target']

# Convert target column to numerical labels
label_encoder = LabelEncoder()
target_encoded = label_encoder.fit_transform(target)

# Standardize the numeric features
numeric_features = features.select_dtypes(include=['int64', 'float64'])
scaler = StandardScaler()
numeric_features_scaled = scaler.fit_transform(numeric_features)

# Combine scaled numeric features with categorical features
features_scaled = pd.concat([pd.DataFrame(numeric_features_scaled, columns=numeric_features.columns), pd.DataFrame(target_encoded, columns=['target'])], axis=1)

# Find optimal number of clusters using silhouette score for numeric features
best_score = -1
best_k = 2  # assuming there are at least 2 clusters
for k in range(2, 6):  # trying different numbers of clusters
    kproto = KPrototypes(n_clusters=k, init='Cao', verbose=2)  # You can adjust the init parameter based on your data
    clusters = kproto.fit_predict(features_scaled.values, categorical=[features_scaled.shape[1] - 1])
    silhouette_avg = silhouette_score(features_scaled, clusters)
    if silhouette_avg > best_score:
        best_score = silhouette_avg
        best_k = k

# Fit K-Prototype with the best number of clusters for numeric features
kproto = KPrototypes(n_clusters=best_k, init='Cao', verbose=2)  # You can adjust the init parameter based on your data
clusters = kproto.fit_predict(features_scaled.values, categorical=[features_scaled.shape[1] - 1])

# Assign cluster labels to the original dataset
df['cluster'] = clusters

# Function to identify cluster for a given data point
def identify_cluster(data_point):
    scaled_point_numeric = scaler.transform([data_point[0:len(numeric_features.columns)]])
    data_point_scaled = pd.concat([pd.DataFrame(scaled_point_numeric, columns=numeric_features.columns), pd.DataFrame([data_point[len(numeric_features.columns):]], columns=['target'])], axis=1)
    cluster_label = kproto.predict(data_point_scaled.values, categorical=[data_point_scaled.shape[1] - 1])
    return cluster_label[0]

# Example usage
user_data_point = [-70, -61, -66, -53, -51, -63, -82, -57, -76, -78, -66, -66, -61, -59, -73, -75, -63, -77, 'B37']
user_data_point_encoded = label_encoder.transform([user_data_point[-1]])[0]  # Encode the target column
user_data_point = user_data_point[:-1] + [user_data_point_encoded]  # Replace the target column with the encoded value
predicted_cluster = identify_cluster(user_data_point)
print(f"The provided data point belongs to Cluster {predicted_cluster}")

# Save the clustered dataset to a new CSV file
df.to_csv('clustered_data.csv', index=False)


Initialization method and algorithm are deterministic. Setting n_init to 1.
Init: initializing centroids
Init: initializing clusters


Starting iterations...
Run: 1, iteration: 1/100, moves: 6340, ncost: 528809.8582013213
Run: 1, iteration: 2/100, moves: 4193, ncost: 508583.3698206701
Run: 1, iteration: 3/100, moves: 1877, ncost: 505194.5931780015
Run: 1, iteration: 4/100, moves: 403, ncost: 505035.5969200729
Run: 1, iteration: 5/100, moves: 56, ncost: 505031.5787013974
Run: 1, iteration: 6/100, moves: 5, ncost: 505031.53580043605
Run: 1, iteration: 7/100, moves: 0, ncost: 505031.53580043605
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 2, iteration: 1/100, moves: 5965, ncost: 529745.1460446548
Run: 2, iteration: 2/100, moves: 3496, ncost: 516815.9784478408
Run: 2, iteration: 3/100, moves: 2331, ncost: 510527.07486296835
Run: 2, iteration: 4/100, moves: 1794, ncost: 506795.0812267524
Run: 2, iteration: 5/100, moves: 1213, ncost: 505041.2069304232
Run: 2, iteration: 6/100, moves: 97, ncost: 505031.5680034098
Run: 2, iteration: 7/100, moves: 4, ncost: 505031.53787501523
Run: 2, ite

In [1]:
!pip install kmodes


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
!pip install flask